<a href="https://colab.research.google.com/github/RamezHatab/Spotify-Suggestion-Model/blob/main/Spotify_Suggestion_Model_Completed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spotify Suggestion Model V3

## Infrastructure

In [8]:
import sys
!{sys.executable} -m pip install spotipy

     |████████████████████████████████| 138 kB 7.6 MB/s 
     |████████████████████████████████| 62 kB 843 kB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [6]:
username = "agwx2"
cid = "17c11cf9271f4980a85e6aab83cba6bb" 
secret = "015c9abf126340a7981e7ba64dbe7828"
# good_playlist_id = "61BmnG6d7ifmj8Dr5tuCBK"
# bad_playlist_ids = "53FPslFSIAGvVyERcNG4J3"
# source_playlist_id = "5ghFHvZmV4FDK5YLeZQRu9"
good_playlist_ids = [
    "37i9dQZF1DX48TTZL62Yht",   # Hip Hop Favourites
    "37i9dQZF1EQnqst5TRi17F",   # Hip Hop Mix
    "37i9dQZF1DX7FY5ma9162x",   # R&B Favourites
    "37i9dQZF1EQoqCH7BwIYb7",   # R&B Mix
    "53FPslFSIAGvVyERcNG4J3",   # Trap/R&B (personal mix)
    # "37i9dQZF1DWWEcRhUVtL8n"    # Indie Pop
]
bad_playlist_ids = [
    "37i9dQZF1EQpj7X7UK8OOF",   # Rock Mix
    "37i9dQZF1DX1kCIzMYtzum",   # EDM
    "37i9dQZF1DXbITWG1ZJKYt",   # Jazz Classics
    "37i9dQZF1EQmPV0vrce2QZ",   # Country Mix
    "37i9dQZF1EQncLwOalG3K7",   # Pop Mix
    "61BmnG6d7ifmj8Dr5tuCBK"    # Classic Rock (personal mix)
]
source_playlist_ids = [
    # "37i9dQZF1DXb57FjYWz00c",   # 80s Hits
    # "37i9dQZF1EQn2GRFTFMl2A",   # 90s Mix
    # "37i9dQZF1DWUZv12GM5cFk",   # 2000s Hits
    # "37i9dQZF1DXc6IFF23C9jj",   # 2010s Hits
    "5ghFHvZmV4FDK5YLeZQRu9",   # Top 100 (personal mix)
]
source_playlist_id = source_playlist_ids[0]

In [9]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

import pandas as pd
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression

from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

## Data Collection

In [24]:
sp_cred = SpotifyClientCredentials(
    client_id=cid,
    client_secret=secret
)
sp = spotipy.Spotify(client_credentials_manager=sp_cred)

In [25]:
all_good_ids = []
for good_playlist_id in good_playlist_ids:
  good_playlist = sp.user_playlist(username, good_playlist_id)
  good_tracks = good_playlist["tracks"]
  good_songs = good_tracks["items"]

  while good_tracks['next']:
      good_tracks = sp.next(good_tracks)
      for item in good_tracks["items"]:
          good_songs.append(item)

  good_ids = []
  for i in range(len(good_songs)):
      good_ids.append(good_songs[i]['track']['id'])
  print(len(good_ids))
  all_good_ids += good_ids
print("")
print(len(all_good_ids))
print(all_good_ids)

100
50
100
50
217

517
['2SAqBLGA283SUiwJ3xOUVI', '1H7KnK26kc1YyellpbINEn', '7o4gBbTM6UBLkOYPw9xMCz', '1VLtjHwRWOVJiE5Py7JxoQ', '35RJhm1pEovTBwnNR0zWad', '5gub2bpJRgJP4m4MliqtdV', '0PJ4dXVkpNAtoOZsEjqktc', '57VeLYXrvNxe8Vs18K2M9W', '0k7wmahjkn389wAZdz19Cv', '3mDFLytDotXo2p0rvfGbkA', '170DNUSnQRp1tGpppF26pA', '1XXimziG1uhM0eDNCZCrUl', '5jbDih9bLGmI8ycUKkN5XA', '0cu0rhnlCnJ7JIeHRiLpj3', '1JXBNvlDEl2epKSJ9EOgKX', '7sO5G9EABYOXQKNPNiE9NR', '6gi6y1xwmVszDWkUqab1qw', '5sBplhIrmzA4kQGyIOpmDf', '4TIqzdAssasqx3DAe6cG9J', '37Nqx7iavZpotJSDXZWbJ3', '18vXApRmJSgQ6wG2ll9AOg', '3w1WjD2zJqjBjDz5fwqQPJ', '0E72UaOjGEOHJliDjq6Ivw', '1ylymaTbSjggG7TaItdRkI', '78QR3Wp35dqAhFEc2qAGjE', '6piAUJJQFD8oHDUr0b7l7q', '26UxwWl9xCb83OynXELJcL', '2Xo3rLHjXfdb4avN68aQyi', '1EWkw4Fa6IlnsAihLUlFFM', '7tTRFVlFzk6yTnoMcParmi', '3McTpsUQfEDrlNd2ikro2E', '1fxYz1s15pyZLsJT0Hdh1m', '6EDO9iiTtwNv6waLwa1UUq', '6wJYhPfqk3KGhHRG76WzOh', '5Z8HZM6iQMhhqyPcCGY5g9', '0C1T6YOwB7MMzspV0UbKKA', '13EW51DKru044OJDJ9Bm8o', '2QpGZOhTCHHiK

In [26]:
all_bad_ids = []
for bad_playlist_id in bad_playlist_ids:
  bad_playlist = sp.user_playlist(username, bad_playlist_id)
  bad_tracks = bad_playlist["tracks"]
  bad_songs = bad_tracks["items"]

  while bad_tracks['next']:
      bad_tracks = sp.next(bad_tracks)
      for item in bad_tracks["items"]:
          bad_songs.append(item)

  bad_ids = []
  for i in range(len(bad_songs)):
      bad_ids.append(bad_songs[i]['track']['id'])
  print(len(bad_ids))
  all_bad_ids += bad_ids
print("")
print(len(all_bad_ids))
print(all_bad_ids)

50
75
200
50
50
132

557
['4UJmPSJsBsIR1U0N79BU1g', '4bHsxqR3GMrXTxEPLuK5ue', '2jz6rWGqVCYsymwxGoaON4', '6N1EjQjnvhOjFrF6oUmGPa', '776AftMmFFAWUIEAb3lHhw', '2RlgNHKcydI9sayD2Df2xp', '4pbJqGIASGPr0ZpGpnWkDn', '6ADSaE87h8Y3lccZlBJdXH', '37Tmv4NnfQeb0ZgUC4fOJj', '1FTSo4v6BOZH9QxKc3MbVM', '0WSa1sucoNRcEeULlZVQXj', '1hwJKpe0BPUsq6UUrwBWTw', '4o6BgsqLIBViaGVbx5rbRk', '3BQHpFgAp4l80e1XslIjNI', '70C4NyhjD5OZUMzvWZ3njJ', '1jzDzZWeSDBg5fhNc3tczV', '6Qyc6fS4DsZjB2mRW9DsQs', '5sICkBXVmaCQk5aISGR3x1', '3gdewACMIVMEWVbyb8O9sY', '5T8EDUDqKcs6OSOwEsfqG7', '4RvWPyQ5RL0ao9LPZeSouE', '5qqabIl2vWzo9ApSC317sa', '4pt5fDVTg5GhEvEtlz9dKk', '07q0QVgO56EorrSGHC48y3', '7EZC6E7UjZe63f1jRmkWxt', '1QFh8OH1e78dGd3VyJZCAC', '003vvx7Niy0yvhvHt4a68B', '7o2CTH4ctstm8TNelqjb51', '57bgtoPSgt236HzfBOd8kj', '4cIPLtg1avt2Jm3ne9S1zy', '1f3yAtsJtY87CTmM8RLnxf', '7BY005dacJkbO6EPiOh2wb', '63T7DJ1AFDD6Bn8VzG6JE8', '2374M0fQpWi3dLnB54qaLX', '3MrRksHupTVEQ7YbA0FsZK', '37ZJ0p5Jm13JPevGcx4SkF', '1RQwcsdHIW5p86cx8zuf1I', '39shmbIHICJ

In [27]:
source_playlist = sp.user_playlist(username, source_playlist_id)
source_tracks = source_playlist["tracks"]
source_songs = source_tracks["items"]

while source_tracks['next']:
    source_tracks = sp.next(source_tracks)
    for item in source_tracks["items"]:
        source_songs.append(item)

source_ids = []
for i in range(len(source_songs)):
    if source_songs[i]['track'] != None:
      source_ids.append(source_songs[i]['track']['id'])
print(len(source_ids))
print(source_ids)

129
['0Dm43YLUlGdePpSbI1ct8h', '70eFcWOvlMObDhURTqT4Fv', '1DhRbox3xkceP64k3JeYfW', '3FHXUqaUer9o231Je1zxIN', '2md2i5QvelRFnafpnd6LOg', '1z3L82jeYnTmYUjPc7zXcu', '1MJ5f5EYBC92ADD6xcz7nb', '0W0R48uImgPVVJgb5YTC3I', '0nMqu5vN44GNv8UOQxmBJK', '5pgZpHqfv4TSomtkfGZGrG', '14rWEXQGBsFB7ALUc8G4sr', '4sqIBV3ukQDqRr8Me2nvQb', '1QvpsbjYAVVFvFrWVC0qnQ', '6Wz9rIfo9tOBcVCd1Mo7F7', '3OUyyDN7EZrL7i0Sbi5SVd', '32q9OuCp7wFdLZKuCXm0wl', '5JWPUEov2wlX7c0jhYZpeB', '2BITQ360Knh6qNAOqR7Dyq', '5O2Yjlcs5pSpHq1gcA0cwg', '23bOfmnZiN42pB1frXNyak', '1LwwZ0xoUz5gR8XrxEUKAK', '17aVK3oQdgwVG2l0HpP67W', '25nzKGDiua1lE9Qo5V19GL', '60dwJ8dnqZ4WrIdiu7LkNR', '7KrlNS2VqiNNJ82RMQdxBn', '2xdwCJLfKCkat14mlgR26M', '3Jbpo2slhW2bbVneiJnC9G', '4dVbhS6OiYvFikshyaQaCN', '5UWwZ5lm5PKu6eKsHAGxOk', '67vYdAAM3oGsEImCRfbtsF', '3ptB4SmuSJTGhQByFrMJVJ', '2CDVqsGGkf2v1tMETt3fq5', '4QMSoa7Fh3y4kmMNuBtOgt', '1F9kEuwT29fgqlovIclu81', '0EC0kuRKIUlru5eRxWYYyv', '6KYVWcOQWm7JrlsPMkrkNi', '732Sqj2piYPhdMPENP4axz', '0ooz46SAYMJsdOPnwW4nKK', '1mSClO

In [28]:
training_features = []

for i in range(0, len(all_good_ids), 50):
  audio_features = sp.audio_features(all_good_ids[i : i + 50])
  for track in audio_features:
    if track != None:

      track['target'] = 1
      training_features.append(track)
      # training_features[-1]['target'] = 1

for i in range(0, len(all_bad_ids), 50):
  audio_features = sp.audio_features(all_bad_ids[i : i + 50])
  for track in audio_features:
    if track != None:
      track['target'] = 0
      training_features.append(track)
      # training_features[-1]['target'] = 0

print(len(training_features))
print(training_features)

1073
[{'danceability': 0.761, 'energy': 0.518, 'key': 0, 'loudness': -8.871, 'mode': 1, 'speechiness': 0.134, 'acousticness': 0.244, 'instrumentalness': 3.47e-05, 'liveness': 0.107, 'valence': 0.522, 'tempo': 133.976, 'type': 'audio_features', 'id': '2SAqBLGA283SUiwJ3xOUVI', 'uri': 'spotify:track:2SAqBLGA283SUiwJ3xOUVI', 'track_href': 'https://api.spotify.com/v1/tracks/2SAqBLGA283SUiwJ3xOUVI', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2SAqBLGA283SUiwJ3xOUVI', 'duration_ms': 261493, 'time_signature': 4, 'target': 1}, {'danceability': 0.49, 'energy': 0.618, 'key': 1, 'loudness': -5.655, 'mode': 1, 'speechiness': 0.104, 'acousticness': 0.0221, 'instrumentalness': 3.77e-06, 'liveness': 0.259, 'valence': 0.286, 'tempo': 99.7, 'type': 'audio_features', 'id': '1H7KnK26kc1YyellpbINEn', 'uri': 'spotify:track:1H7KnK26kc1YyellpbINEn', 'track_href': 'https://api.spotify.com/v1/tracks/1H7KnK26kc1YyellpbINEn', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/1H7KnK26kc1Yye

In [29]:
source_features = []
for i in range(0, len(source_ids), 50):
    audio_features = sp.audio_features(source_ids[i : i + 50])
    for track in audio_features:
        track['target'] = 0
        source_features.append(track)
        # source_features[-1]['target'] = 0  # arbitrary

print(len(source_features))
print(source_features)

129
[{'danceability': 0.662, 'energy': 0.624, 'key': 10, 'loudness': -13.606, 'mode': 0, 'speechiness': 0.0417, 'acousticness': 0.165, 'instrumentalness': 0, 'liveness': 0.0674, 'valence': 0.961, 'tempo': 174.513, 'type': 'audio_features', 'id': '0Dm43YLUlGdePpSbI1ct8h', 'uri': 'spotify:track:0Dm43YLUlGdePpSbI1ct8h', 'track_href': 'https://api.spotify.com/v1/tracks/0Dm43YLUlGdePpSbI1ct8h', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0Dm43YLUlGdePpSbI1ct8h', 'duration_ms': 252560, 'time_signature': 4, 'target': 0}, {'danceability': 0.64, 'energy': 0.647, 'key': 5, 'loudness': -8.114, 'mode': 0, 'speechiness': 0.186, 'acousticness': 0.124, 'instrumentalness': 0, 'liveness': 0.0801, 'valence': 0.546, 'tempo': 92.979, 'type': 'audio_features', 'id': '70eFcWOvlMObDhURTqT4Fv', 'uri': 'spotify:track:70eFcWOvlMObDhURTqT4Fv', 'track_href': 'https://api.spotify.com/v1/tracks/70eFcWOvlMObDhURTqT4Fv', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/70eFcWOvlMObDhURTqT4Fv'

## Data Preparation

In [30]:
trainingData = pd.DataFrame(training_features)

In [31]:
trainingData

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,target
0,0.761,0.518,0,-8.871,1,0.1340,0.24400,0.000035,0.107,0.522,133.976,audio_features,2SAqBLGA283SUiwJ3xOUVI,spotify:track:2SAqBLGA283SUiwJ3xOUVI,https://api.spotify.com/v1/tracks/2SAqBLGA283S...,https://api.spotify.com/v1/audio-analysis/2SAq...,261493,4,1
1,0.490,0.618,1,-5.655,1,0.1040,0.02210,0.000004,0.259,0.286,99.700,audio_features,1H7KnK26kc1YyellpbINEn,spotify:track:1H7KnK26kc1YyellpbINEn,https://api.spotify.com/v1/tracks/1H7KnK26kc1Y...,https://api.spotify.com/v1/audio-analysis/1H7K...,201600,4,1
2,0.928,0.341,11,-10.196,0,0.1030,0.06140,0.000000,0.162,0.745,129.966,audio_features,7o4gBbTM6UBLkOYPw9xMCz,spotify:track:7o4gBbTM6UBLkOYPw9xMCz,https://api.spotify.com/v1/tracks/7o4gBbTM6UBL...,https://api.spotify.com/v1/audio-analysis/7o4g...,170233,4,1
3,0.832,0.534,1,-7.813,0,0.2530,0.34500,0.000000,0.165,0.116,93.924,audio_features,1VLtjHwRWOVJiE5Py7JxoQ,spotify:track:1VLtjHwRWOVJiE5Py7JxoQ,https://api.spotify.com/v1/tracks/1VLtjHwRWOVJ...,https://api.spotify.com/v1/audio-analysis/1VLt...,132833,4,1
4,0.695,0.343,5,-10.798,1,0.0432,0.07430,0.000001,0.107,0.229,85.840,audio_features,35RJhm1pEovTBwnNR0zWad,spotify:track:35RJhm1pEovTBwnNR0zWad,https://api.spotify.com/v1/tracks/35RJhm1pEovT...,https://api.spotify.com/v1/audio-analysis/35RJ...,218902,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1068,0.547,0.987,8,-3.146,1,0.0839,0.00191,0.006190,0.322,0.901,79.976,audio_features,6HjE9DWhhPLSlmIMEyZzRc,spotify:track:6HjE9DWhhPLSlmIMEyZzRc,https://api.spotify.com/v1/tracks/6HjE9DWhhPLS...,https://api.spotify.com/v1/audio-analysis/6HjE...,178627,4,0
1069,0.469,0.537,9,-10.421,1,0.1520,0.02410,0.002010,0.269,0.783,124.193,audio_features,3aoDEt6zSuYQ47gzarlaVo,spotify:track:3aoDEt6zSuYQ47gzarlaVo,https://api.spotify.com/v1/tracks/3aoDEt6zSuYQ...,https://api.spotify.com/v1/audio-analysis/3aoD...,382834,1,0
1070,0.322,0.265,2,-13.398,1,0.0302,0.07260,0.340000,0.366,0.207,133.577,audio_features,2gCf9PqIvpKbxwfJMBWwbn,spotify:track:2gCf9PqIvpKbxwfJMBWwbn,https://api.spotify.com/v1/tracks/2gCf9PqIvpKb...,https://api.spotify.com/v1/audio-analysis/2gCf...,226667,4,0
1071,0.373,0.453,9,-9.224,1,0.0419,0.26100,0.000046,0.160,0.418,95.384,audio_features,4nla8cAvm6AQFnNou9WiR1,spotify:track:4nla8cAvm6AQFnNou9WiR1,https://api.spotify.com/v1/tracks/4nla8cAvm6AQ...,https://api.spotify.com/v1/audio-analysis/4nla...,196841,4,0


In [32]:
train, test = train_test_split(trainingData, test_size = 0.15)

In [33]:
target_features = ["danceability", "loudness", "valence", "energy", "instrumentalness", "acousticness", "key", "speechiness", "duration_ms"]

x_train = train[target_features]
y_train = train["target"]
x_test = test[target_features]
y_test = test["target"]

In [34]:
x_train

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
816,0.476,-19.166,0.118,0.0794,0.872000,0.955000,9,0.0365,162680
897,0.672,-6.446,0.280,0.5260,0.000001,0.031100,4,0.0599,143747
909,0.652,-5.714,0.483,0.5570,0.000000,0.740000,1,0.0318,169354
990,0.338,-11.448,0.653,0.8310,0.000008,0.013100,11,0.0649,246187
793,0.277,-18.442,0.285,0.2420,0.464000,0.694000,4,0.0342,824133
...,...,...,...,...,...,...,...,...,...
202,0.798,-7.383,0.260,0.5280,0.000041,0.073600,7,0.0396,202777
1032,0.431,-6.011,0.365,0.9470,0.000253,0.000779,9,0.0752,218867
1012,0.292,-4.459,0.423,0.8380,0.479000,0.001400,9,0.0780,333800
356,0.695,-8.627,0.210,0.6280,0.219000,0.337000,5,0.1450,244653


In [35]:
y_train

816     0
897     0
909     0
990     0
793     0
       ..
202     1
1032    0
1012    0
356     1
974     0
Name: target, Length: 912, dtype: int64

In [36]:
x_test

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
1025,0.405,-5.478,0.410,0.7720,0.079600,0.000801,9,0.0365,332440
426,0.521,-6.750,0.264,0.7620,0.000011,0.139000,2,0.5340,433800
1015,0.251,-5.002,0.482,0.7300,0.013700,0.024000,6,0.0615,387960
960,0.564,-10.068,0.619,0.9320,0.169000,0.013000,11,0.0554,146250
91,0.762,-8.105,0.280,0.5830,0.491000,0.232000,11,0.0407,128517
...,...,...,...,...,...,...,...,...,...
168,0.617,-10.426,0.377,0.5250,0.000001,0.554000,8,0.1200,212867
773,0.443,-20.146,0.596,0.0711,0.894000,0.989000,1,0.0486,172293
975,0.290,-6.139,0.101,0.9370,0.104000,0.000150,9,0.1000,351707
710,0.470,-7.873,0.483,0.6840,0.000019,0.714000,4,0.0377,399467


In [37]:
y_test

1025    0
426     1
1015    0
960     0
91      1
       ..
168     1
773     0
975     0
710     0
810     0
Name: target, Length: 161, dtype: int64

## Model Evaluation

### Random Forest

In [38]:
# random forest combines multiple decision trees and uses bagging
# (training multiple trees on different sections of training data, averaging the result)
rfc = RandomForestClassifier(n_jobs=1, random_state=1)
rfc.fit(x_train, y_train)
rfc_pred = rfc.predict(x_test)

score = accuracy_score(y_test, rfc_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, rfc_pred))

Accuracy using Random Forest:  85.71 %
Mean squared error: 0.14


Measured Accuracy: 88.82%

### Gradient Boosting

In [39]:
# I <3 gradient boost
gbc = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)
gbc.fit(x_train, y_train)
gbc_pred = gbc.predict(x_test)

score = accuracy_score(y_test, gbc_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, gbc_pred))

Accuracy using Gradient Boost:  86.34 %
Mean squared error: 0.14


Measured Accuracy: 90.68%

### Gradient Boosting w/ Manual Stacking (Random Forest)

In [40]:
x_train_rfc = (x_train.iloc[0:int(len(x_train) / 2)]).copy()
y_train_rfc = (y_train.iloc[0:int(len(y_train) / 2)]).copy()
x_train_gbc = (x_train.iloc[int(len(x_train) / 2):len(x_train)]).copy()
y_train_gbc = (y_train.iloc[int(len(y_train) / 2):len(y_train)]).copy()

In [41]:
x_test_rfc = x_train_gbc
y_test_rfc = y_train_gbc
x_test_gbc = x_test.copy()
y_test_gbc = y_test.copy()

In [43]:
rfc_stack = RandomForestClassifier(n_jobs=1, random_state=1)
rfc_stack.fit(x_train_rfc, y_train_rfc)
rfc_stack_pred = rfc_stack.predict(x_test_rfc)

score = accuracy_score(y_test_rfc, rfc_stack_pred) * 100
print("Accuracy using Random Forest: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test_rfc, rfc_stack_pred))

Accuracy using Random Forest:  85.53 %
Mean squared error: 0.14


In [44]:
rfc_stack_pred_test = rfc_stack.predict(x_test_gbc)

In [45]:
# stack
x_train_gbc['rfc_stack_pred'] = rfc_stack_pred_train.tolist()
x_test_gbc['rfc_stack_pred'] = rfc_stack_pred_test.tolist()

In [46]:
x_train_gbc

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms,rfc_stack_pred
534,0.577,-8.691,0.604,0.828,0.011400,0.002130,6,0.0298,331267,0
533,0.315,-8.072,0.497,0.715,0.000008,0.001250,11,0.0362,289533,0
143,0.843,-7.247,0.921,0.538,0.000000,0.483000,10,0.0534,140480,1
692,0.549,-13.793,0.320,0.124,0.001750,0.938000,0,0.0397,360827,0
776,0.444,-16.936,0.656,0.339,0.103000,0.660000,3,0.0418,456360,0
...,...,...,...,...,...,...,...,...,...,...
202,0.798,-7.383,0.260,0.528,0.000041,0.073600,7,0.0396,202777,1
1032,0.431,-6.011,0.365,0.947,0.000253,0.000779,9,0.0752,218867,0
1012,0.292,-4.459,0.423,0.838,0.479000,0.001400,9,0.0780,333800,0
356,0.695,-8.627,0.210,0.628,0.219000,0.337000,5,0.1450,244653,1


In [47]:
x_test_gbc

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms,rfc_stack_pred
1025,0.405,-5.478,0.410,0.7720,0.079600,0.000801,9,0.0365,332440,0
426,0.521,-6.750,0.264,0.7620,0.000011,0.139000,2,0.5340,433800,1
1015,0.251,-5.002,0.482,0.7300,0.013700,0.024000,6,0.0615,387960,0
960,0.564,-10.068,0.619,0.9320,0.169000,0.013000,11,0.0554,146250,0
91,0.762,-8.105,0.280,0.5830,0.491000,0.232000,11,0.0407,128517,1
...,...,...,...,...,...,...,...,...,...,...
168,0.617,-10.426,0.377,0.5250,0.000001,0.554000,8,0.1200,212867,1
773,0.443,-20.146,0.596,0.0711,0.894000,0.989000,1,0.0486,172293,0
975,0.290,-6.139,0.101,0.9370,0.104000,0.000150,9,0.1000,351707,0
710,0.470,-7.873,0.483,0.6840,0.000019,0.714000,4,0.0377,399467,0


In [48]:
gbc_stack = GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)
gbc_stack.fit(x_train_gbc, y_train_gbc)
gbc_stack_pred = gbc_stack.predict(x_test_gbc)

score = accuracy_score(y_test_gbc, gbc_stack_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test_gbc, gbc_stack_pred))

Accuracy using Gradient Boost:  85.09 %
Mean squared error: 0.15


Measured Accuracy: 88.2%

### Stacking Classifier

In [51]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
]

stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
stk.fit(x_train, y_train)
stk_pred = stk.predict(x_test)

score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))

Accuracy using Gradient Boost:  86.34 %
Mean squared error: 0.14


Measured Accuracy: 88.2%

In [52]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1))
]

stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
stk.fit(x_train, y_train)
stk_pred = stk.predict(x_test)

score = accuracy_score(y_test, stk_pred) * 100
print("Accuracy using Gradient Boost: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, stk_pred))

Accuracy using Gradient Boost:  85.09 %
Mean squared error: 0.15


Measured Accuracy: 89.44%

### Neural Networks

#### Baseline

In [53]:
def keras_nn_baseline_model_small():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [54]:
def keras_nn_baseline_model_big():
  model = Sequential()
  model.add(Dense(9, input_dim=9, activation='relu'))
  model.add(Dense(18, input_dim=9, activation='relu'))
  model.add(Dense(90, input_dim=18, activation='sigmoid'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

In [55]:
def keras_nn_baseline_model_shaped():
  model = Sequential()
  model.add(Dense(5, input_dim=9, activation='relu'))
  model.add(Dense(60, input_dim=5, activation='sigmoid'))
  model.add(Dense(5, input_dim=60, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model

#### Pipeline w/ Normalization

In [56]:
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=keras_nn_baseline_model_shaped, epochs=200, batch_size=5, verbose=0)))
keras_pipeline = Pipeline(estimators)

In [57]:
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(keras_pipeline, x_train, y_train, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean() * 100, results.std() * 100))

Standardized: 84.24% (10.68%)


**Cross-Validation Scores**  
keras_nn_baseline_model_small: 87.51%  
keras_nn_baseline_model_big: 85.09%  
keras_nn_baseline_model_shaped: 88.27%  

In [58]:
keras_pipeline.fit(x_train, y_train)
keras_pred = keras_pipeline.predict(x_test)

score = accuracy_score(y_test, keras_pred) * 100
print("Accuracy using NN: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, keras_pred))

Accuracy using NN:  85.71 %
Mean squared error: 0.14


### Neural Networks w/ Stacking

In [59]:
def get_nn_classifier():
  nn_clf = KerasClassifier(build_fn=keras_nn_baseline_model_shaped, epochs=200, batch_size=5, verbose=0)
  nn_clf._estimator_type = "classifier"
  estimators = []
  estimators.append(('standardize', StandardScaler()))
  estimators.append(('mlp', nn_clf))
  return Pipeline(estimators)

In [60]:
base_learners = [
  ('rf_1', RandomForestClassifier(n_estimators=10, random_state=42)),
  ('rf_2', KNeighborsClassifier(n_neighbors=5)),             
  ('rf_3', GradientBoostingClassifier(n_estimators=100, learning_rate=.1, max_depth=1, random_state=1)),
  ('rf_4', get_nn_classifier())
]

nn_stk = StackingClassifier(estimators=base_learners, final_estimator=LogisticRegression())
nn_stk.fit(x_train, y_train)
nn_stk_pred = nn_stk.predict(x_test)

score = accuracy_score(y_test, nn_stk_pred) * 100
print("Accuracy using NN & Stacking: ", round(score, 2), "%")
print('Mean squared error: %.2f' % mean_squared_error(y_test, nn_stk_pred))

Accuracy using NN & Stacking:  87.58 %
Mean squared error: 0.12


Measured Accuracy: 90.06%

## Generate Predictions

In [61]:
sourceData = pd.DataFrame(source_features)
sourceData = sourceData[target_features]

In [62]:
sourceData

,danceability,loudness,valence,energy,instrumentalness,acousticness,key,speechiness,duration_ms
0,0.662,-13.606,0.961,0.624,0.000000,0.16500,10,0.0417,252560
1,0.640,-8.114,0.546,0.647,0.000000,0.12400,5,0.1860,197867
2,0.716,-5.313,0.910,0.823,0.000000,0.13200,5,0.0495,161747
3,0.468,-6.536,0.762,0.630,0.000000,0.31000,8,0.0843,204507
4,0.397,-4.577,0.556,0.903,0.000002,0.00575,2,0.0457,201373
...,...,...,...,...,...,...,...,...,...
124,0.625,-8.337,0.556,0.686,0.049900,0.01540,9,0.0309,211693
125,0.192,-12.456,0.108,0.389,0.930000,0.92100,2,0.0346,260135
126,0.292,-7.676,0.417,0.690,0.154000,0.12300,7,0.0311,262091
127,0.552,-6.595,0.588,0.645,0.000001,0.00187,7,0.0268,299733


In [63]:
# model = gbc
# model = keras_pipeline
model = nn_stk

In [64]:
pred = model.predict(sourceData)

In [67]:
source_playlist_songs[0]

{'added_at': '2020-10-01T05:24:42Z',
 'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/agwx2'},
  'href': 'https://api.spotify.com/v1/users/agwx2',
  'id': 'agwx2',
  'type': 'user',
  'uri': 'spotify:user:agwx2'},
 'is_local': False,
 'primary_color': None,
 'track': None,
 'video_thumbnail': None}

In [66]:

source_playlist_songs = source_playlist['tracks']['items']

print("ACCEPTED")


i = 0
for prediction in pred:
    if prediction == 1 and i + 1 < len(source_playlist_songs):
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1


print("REJECTED")

i = 0
for prediction in pred:
    if prediction == 0 and i + 1 < len(source_playlist_songs):
        song_title = source_playlist_songs[i + 1]['track']['name']
        artist_name = source_playlist_songs[i + 1]['track']['artists'][0]['name']
        print ("Song: " + song_title + ", By: "+ artist_name)
    i = i + 1

ACCEPTED
Song: Beautiful People (feat. Khalid), By: Ed Sheeran
Song: I Don't Want Your Money (feat. H.E.R.), By: Ed Sheeran
Song: Revolution 909, By: Daft Punk
Song: Twenty Something, By: Nightly
Song: That's Life, By: Still Woozy
Song: Order More (feat. Starrah), By: G-Eazy
Song: Paper Trail$, By: Joey Bada$$
Song: Baby, By: Brandy
Song: I Like Dat, By: T-Pain
Song: Lost Cause, By: Billie Eilish
Song: STUNNAMAN (with Roddy Ricch & feat. Lil Wayne), By: Birdman
Song: Baddest (feat. Chris Brown & 2 Chainz), By: Yung Bleu
Song: Y U DON'T LOVE ME? (MISS AMERIKKKA), By: Joey Bada$$
Song: Easy Love, By: Lauv
Song: DRUGS, By: Tai Verdes
Song: Loser, By: Beck
Song: Run To The Sun, By: N.E.R.D
Song: Crush, By: Souly Had
Song: If I Got It (Your Love Brought It), By: Aaron Frazer
Song: Didn't Cha Know, By: Erykah Badu
Song: Red (Brothel Remix), By: Lucy Daydream
Song: Rocky, By: Still Woozy
Song: Butterflies, By: Michael Jackson
Song: Bake Sale (feat. Travis Scott), By: Wiz Khalifa
Song: CASINO,